# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd

from plotting import *
from prices import *
from imputation import *
from inflation import *

In [3]:
results_dir = 'results/refactor_imputation/'
os.makedirs(results_dir, exist_ok=True)

# Dataset

## ML systems

In [4]:
pcd_df = pd.read_csv('data/All ML Systems - full view.csv')
pcd_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Organization (from Organization),Base model,Finetune compute (FLOP),Finetune compute notes,Authors by country,Hardware quantity,Hardware utilization,Training cost trends,Training cloud compute vendor,Training data center
0,Cohere Command,Language,NaN,NaN,NaN,NaN,NaN,https://cohere.com/models/command,NaN,"World-class AI, at your command",...,Cohere,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Theseus,Other,Maze solving,Claude Shannon,Historical significance,NaN,NaN,https://www.technologyreview.com/2018/12/19/13...,0.0,Mighty Mouse,...,Bell Laboratories,NaN,NaN,NaN,Theseus,NaN,NaN,NaN,NaN,NaN
2,SNARC,Other,Maze solving,Marvin Minsky,Historical significance,NaN,NaN,https://en.wikipedia.org/wiki/Stochastic_neura...,33.0,A Neural-Analogue Calculator Based upon a Prob...,...,Harvard University,NaN,NaN,NaN,SNARC,NaN,NaN,NaN,NaN,NaN
3,Genetic algorithm,NaN,NaN,NA Barricelli,Historical significance,Possibly first computer simulation of a geneti...,NaN,https://link.springer.com/article/10.1007/BF01...,266.0,Numerical testing of evolution theories,...,Institute for Advanced Study,NaN,NaN,NaN,Genetic algorithm,NaN,NaN,NaN,NaN,NaN
4,Sequence-based pattern recognition,Vision,Character recognition,O. G. Selfridge,Historical significance,NaN,NaN,https://dl.acm.org/doi/10.1145/1455292.1455310,290.0,Pattern recognition and modern computers,...,Massachusetts Institute of Technology (MIT),NaN,NaN,NaN,Sequence-based pattern recognition,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,Mamba-24M (SC09),Speech,Audio generation,"Albert Gu, Tri Dao",SOTA Improvement,"""SC09 is a benchmark speech generation dataset...",NaN,https://arxiv.org/abs/2312.00752,2.0,Mamba: Linear-Time Sequence Modeling with Sele...,...,"Carnegie Mellon University (CMU), Princeton Un...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1131,Mamba-2.8B,Language,Language Generation,"Albert Gu, Tri Dao",NaN,NaN,Permissive license,https://arxiv.org/abs/2312.00752,2.0,Mamba: Linear-Time Sequence Modeling with Sele...,...,"Carnegie Mellon University (CMU), Princeton Un...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1132,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA Improvement,""" Evaluation on a broad range of benchmarks sh...",Unreleased,https://storage.googleapis.com/deepmind-media/...,0.0,Gemini: A Family of Highly Capable Multimodal ...,...,Google DeepMind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1133,Phi-2,Language,Language Generation,"Marah Abdin, Jyoti Aneja, Sebastien Bubeck, Ca...",NaN,"""With only 2.7 billion parameters, Phi-2 surpa...",API accessible,https://www.microsoft.com/en-us/research/blog/...,NaN,Phi-2: The surprising power of small language ...,...,Microsoft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Publication date in datetime format
pcd_df.dropna(subset=['Publication date'], inplace=True)
pcd_df['Publication date'] = pd.to_datetime(pcd_df['Publication date'])

In [6]:
# Manually copied from "Training cost trends" Airtable
frontier_systems = [
    "PaLM 2",
    "GPT-4",
    "Minerva (540B)",
    "Megatron-Turing NLG 530B",
    "GPT-3 175B (davinci)",
    "Meena",
    "AlphaStar",
    "AlphaGo Zero",
    "AlphaGo Master",
    "GNMT",
    "Claude 2",
    "PaLM (540B)",
#     "ERNIE 3.0 Titan",
    "Gopher (280B)",
    "OpenAI Five",
    "T5-11B",
    "Megatron-BERT",
    "ResNeXt-101 32x48d",
    "AlphaZero",
    "Falcon 180B",
    "GPT-3.5 (text-davinci-003)",
    "Chinchilla",
    "Yuan 1.0",
    "Turing-NLG",
    "BigGAN-deep 512x512",
    "NASv3 (CIFAR-10)",
    "AlphaGo Lee",
    "AlphaGo Fan",
    "OPT-175B",
#     "AlphaCode",
    "GLaM",
    "OpenAI Five Rerun",
    "T5-3B",
    "Megatron-LM (8.3B)",
    "FTW",
    "AmoebaNet-A (F=448)",
    "OpenAI TI7 DOTA 1v1",
    "JFT",
    "Llama 2-70B",
    "LLaMA-65B",
    "LaMDA",
    "ALIGN",
    "GShard (dense)",
    "RoBERTa Large",
    "IMPALA",
]

In [7]:
frontier_pcd_df = pcd_df[pcd_df['System'].isin(frontier_systems)]
frontier_pcd_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Organization (from Organization),Base model,Finetune compute (FLOP),Finetune compute notes,Authors by country,Hardware quantity,Hardware utilization,Training cost trends,Training cloud compute vendor,Training data center
265,AlphaGo Fan,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",SOTA improvement,NaN,NaN,https://www.nature.com/articles/nature24270.ep...,14389.0,Mastering the game of Go with deep neural netw...,...,Google DeepMind,NaN,NaN,NaN,AlphaGo Fan,NaN,NaN,AlphaGo Fan,NaN,NaN
275,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14389.0,Mastering the game of Go with deep neural netw...,...,DeepMind,NaN,NaN,NaN,AlphaGo Lee,NaN,NaN,AlphaGo Lee,NaN,NaN
306,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,5948.0,Google's Neural Machine Translation System: Br...,...,Google,NaN,NaN,NaN,GNMT,96.0,NaN,GNMT,NaN,NaN
317,NASv3 (CIFAR-10),Vision,NaN,"Barret Zoph, Quoc V. Le",Highly cited,NaN,NaN,https://arxiv.org/abs/1611.01578,4569.0,Neural Architecture Search with Reinforcement ...,...,Google Brain,NaN,NaN,NaN,NASv3 (CIFAR-10),800.0,NaN,NASv3 (CIFAR-10),NaN,NaN
337,AlphaGo Master,Games,Go,"D Silver, J Schrittwieser, K Simonyan, I Anton...",Highly cited,NaN,NaN,https://www.researchgate.net/publication/32047...,7831.0,Mastering the game of Go without human knowledge,...,DeepMind,NaN,NaN,NaN,AlphaGo Master,NaN,NaN,AlphaGo Master,NaN,NaN
359,JFT,Vision,NaN,"Chen Sun, Abhinav Shrivastava, Saurabh Singh, ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1707.02968,1923.0,Revisiting Unreasonable Effectiveness of Data ...,...,"Google Research, Carnegie Mellon University (CMU)",NaN,NaN,NaN,JFT,50.0,NaN,JFT,NaN,NaN
369,OpenAI TI7 DOTA 1v1,Games,DOTA,NaN,"SOTA Improvement,Historical significance",NaN,NaN,https://openai.com/research/dota-2,0.0,Dota 2,...,OpenAI,NaN,NaN,NaN,OpenAI TI7 DOTA 1v1,NaN,NaN,OpenAI TI7 DOTA 1v1,NaN,NaN
391,AlphaGo Zero,Games,Go,"D Silver, J Schrittwieser, K Simonyan, I Anton...",Highly cited,NaN,NaN,https://www.researchgate.net/publication/32047...,7831.0,Mastering the game of Go without human knowledge,...,DeepMind,NaN,NaN,NaN,AlphaGo Zero,NaN,NaN,AlphaGo Zero,NaN,NaN
403,AlphaZero,Games,NaN,"D Silver, T Hubert, J Schrittwieser, I Antonoglou",Highly cited,NaN,NaN,https://arxiv.org/abs/1712.01815,1343.0,Mastering Chess and Shogi by Self-Play with a ...,...,DeepMind,NaN,NaN,NaN,AlphaZero,64.0,NaN,AlphaZero,NaN,NaN
411,AmoebaNet-A (F=448),Vision,Image classification,"Esteban Real, Alok Aggarwal, Yanping Huang, Qu...",Highly cited,NaN,NaN,https://arxiv.org/abs/1802.01548,2505.0,Regularized Evolution for Image Classifier Arc...,...,Google Brain,NaN,NaN,NaN,AmoebaNet-A (F=448),450.0,NaN,AmoebaNet-A (F=448),NaN,NaN


In [8]:
assert len(frontier_pcd_df) == len(frontier_systems)

## Prices

In [9]:
price_df = pd.read_csv('data/Hardware prices.csv')
price_df.head()

,Price source,Price date,Hardware model,Manufacturer (from Hardware model),Vendor,Location,Price per chip-hour (on-demand),Price per chip-hour (1-year CUD),Price per chip-hour (3-year CUD),Price (hardware purchase)
0,https://web.archive.org/web/20181009102635/htt...,2018-10-09,Google TPU v2,Google,Google Cloud,US,$1.13,NaN,NaN,NaN
1,https://web.archive.org/web/20181011013513/htt...,2018-10-11,Google TPU v3,Google,Google Cloud,US,$2.00,NaN,NaN,NaN
2,https://web.archive.org/web/20181011013513/htt...,2018-10-11,Google TPU v3,Google,Google Cloud,Europe,$2.20,NaN,NaN,NaN
3,https://web.archive.org/web/20190701021000/htt...,2019-07-01,Google TPU v3,Google,Google Cloud,Iowa (us-central1),$2.00,$1.26,$0.90,NaN
4,https://web.archive.org/web/20190728061708/htt...,2019-07-28,Google TPU v3,Google,Google Cloud,Netherlands (europe-west4),$2.00,$1.26,$0.90,NaN


In [10]:
# Price date in datetime format
price_df.dropna(subset=['Price date'], inplace=True)
price_df['Price date'] = pd.to_datetime(price_df['Price date'])

In [11]:
pcd_hardware_model_colname = 'Name of the hardware (from Training hardware)'

## Hardware data

In [12]:
hardware_df = pd.read_csv('data/Chip dataset-Grid view.csv')
hardware_df.head()

,Name of the hardware,Manufacturer,Type,Release date,Release price (USD),FP64 (double precision) Performance (FLOP/s),FP32 (single precision) Performance (FLOP/s),FP16 (half precision) Performance (FLOP/s),Tensor Float 32 (TF32),FP16 Tensor Core,...,Foundry,Number of transistors in million,Prominent Years of usage,Google Cloud pricing ($ per hour) data from 17 dec 2022,Link to datasheet,Source for the Price,ALL ML SYSTEMS,All ML Systems copy,All ML Systems copy.1,Hardware prices
0,3dfx Spectre 1000,Other,GPU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3dfx Spectre 2000,Other,GPU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3dfx Spectre 3000,Other,GPU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3dfx Voodoo4 4000 AGP,Other,GPU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3dfx Voodoo4 4500 AGP,Other,GPU,2000-10-13,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Imputation

In [13]:
# Check original data with missing values
frontier_pcd_df[['System', 'Training hardware', 'Training time (hours)', 'Hardware quantity', 'Hardware utilization']]

,System,Training hardware,Training time (hours),Hardware quantity,Hardware utilization
265,AlphaGo Fan,NaN,NaN,NaN,NaN
275,AlphaGo Lee,NaN,NaN,NaN,NaN
306,GNMT,NVIDIA Tesla K80,4320.0,96.0,NaN
317,NASv3 (CIFAR-10),NaN,NaN,800.0,NaN
337,AlphaGo Master,Google TPU v1,NaN,NaN,NaN
359,JFT,NVIDIA Tesla K80,1440.0,50.0,NaN
369,OpenAI TI7 DOTA 1v1,NaN,NaN,NaN,NaN
391,AlphaGo Zero,Google TPU v1,480.0,NaN,NaN
403,AlphaZero,Google TPU v2,NaN,64.0,NaN
411,AmoebaNet-A (F=448),NVIDIA Tesla K40s,168.0,450.0,NaN


Use k nearest neighbors

In [14]:
# drop unneeded columns from frontier_pcd_df
irrelevant_columns = ['Notability criteria', 'Notability criteria notes', 'Link', 'Citations', 'Parameters notes',
                      'Training compute notes', 'Training dataset notes', 'Dataset size notes',
                      'Inference compute notes', 'Approach', 'Confidence', 'Last modified', 'Created By', 'Benchmark data',
                      'Exclude', 'Authors by country', 'Training cost trends', 'Abstract', 'Compute cost notes',
                      'Training time notes', 'Authors', 'Name of the hardware (from Training hardware)',
                      'Training compute cost (2020 USD)', 'Organization categorization',
                      'Training dataset', 'Inference compute (FLOP)', 'Compute sponsor categorization',
                      'Finetune compute notes']
frontier_pcd_df = frontier_pcd_df.drop(columns=irrelevant_columns)

In [15]:
# fill column 'Training cloud compute vendor' using org_to_cloud_vendor dictionary
org_to_cloud_vendor = {
    'Google': 'Google Cloud',
    'DeepMind': 'Google Cloud',
    'Google DeepMind': 'Google Cloud',
    'Google Brain': 'Google Cloud',
    'Microsoft': 'Microsoft Azure',
    'OpenAI': 'Microsoft Azure',
}
frontier_pcd_df['Training cloud compute vendor'] = frontier_pcd_df['Organization (from Organization)'].map(org_to_cloud_vendor)
frontier_pcd_df['Training cloud compute vendor'] = frontier_pcd_df['Training cloud compute vendor'].fillna('Amazon Web Services')

In [ ]:
# TODO: logarithmic scaling of big-valued columns (just copy this from validate_imputation.ipynb)

In [16]:
# convert datetime to float so that it can be used in kNN
frontier_pcd_df['Publication date'] = datetime_to_float(pcd_df['Publication date'])

# set the System column as the index for formatting purposes
frontier_pcd_df = frontier_pcd_df.set_index('System')

In [17]:
# kNN parameters
num_neighbors_general = 5  # for training time, hardware quantity, utilization
num_neighbors_training_hardware = 5

In [18]:
imputed_pcd_df = knn_impute_numerical_pcd_data(frontier_pcd_df, num_neighbors=num_neighbors_general)

# Impute training hardware separately, because it is a categorical variable
# There could be a better solution to this, but it seems complicated no matter what - see https://stackoverflow.com/questions/64900801/implementing-knn-imputation-on-categorical-variables-in-an-sklearn-pipeline
imputed_pcd_df = knn_impute_categorical_column(
    imputed_pcd_df,
    num_neighbors=num_neighbors_training_hardware,
    target_col='Training hardware'
)

# Restore the System column
imputed_pcd_df['System'] = frontier_pcd_df.index

# set the System column as the index
imputed_pcd_df = imputed_pcd_df.set_index('System')

# insert imputed values into frontier_pcd_df
frontier_pcd_df['Training hardware'] = imputed_pcd_df['Training hardware']
frontier_pcd_df['Hardware quantity'] = imputed_pcd_df['Hardware quantity']
frontier_pcd_df['Hardware utilization'] = imputed_pcd_df['Hardware utilization']
frontier_pcd_df['Training time (hours)'] = imputed_pcd_df['Training time (hours)']
# calculate training time (chip hours) from training time and hardware quantity
# TODO: try estimating this from compute and FLOP/s instead. Compare the results.
frontier_pcd_df['Training time (chip hours)'] = frontier_pcd_df['Training time (hours)'] * frontier_pcd_df['Hardware quantity']

In [19]:
imputed_pcd_df[['Training hardware', 'Training time (hours)', 'Hardware quantity', 'Hardware utilization']]

,Training hardware,Training time (hours),Hardware quantity,Hardware utilization
System,,,,
AlphaGo Fan,Google TPU v3,375.20,311.4,0.30008
AlphaGo Lee,Google TPU v3,375.20,311.4,0.30008
GNMT,NVIDIA Tesla K80,4320.00,96.0,0.32308
NASv3 (CIFAR-10),Google TPU v3,375.20,800.0,0.30008
AlphaGo Master,Google TPU v1,2073.98,896.0,0.30008
JFT,NVIDIA Tesla K80,1440.00,50.0,0.32308
OpenAI TI7 DOTA 1v1,Google TPU v3,375.20,311.4,0.30008
AlphaGo Zero,Google TPU v1,480.00,3040.0,0.43608
AlphaZero,Google TPU v2,1306.44,64.0,0.30008


In [20]:
frontier_pcd_df[['Training hardware', 'Training time (hours)', 'Hardware quantity', 'Hardware utilization']]

,Training hardware,Training time (hours),Hardware quantity,Hardware utilization
System,,,,
AlphaGo Fan,Google TPU v3,375.20,311.4,0.30008
AlphaGo Lee,Google TPU v3,375.20,311.4,0.30008
GNMT,NVIDIA Tesla K80,4320.00,96.0,0.32308
NASv3 (CIFAR-10),Google TPU v3,375.20,800.0,0.30008
AlphaGo Master,Google TPU v1,2073.98,896.0,0.30008
JFT,NVIDIA Tesla K80,1440.00,50.0,0.32308
OpenAI TI7 DOTA 1v1,Google TPU v3,375.20,311.4,0.30008
AlphaGo Zero,Google TPU v1,480.00,3040.0,0.43608
AlphaZero,Google TPU v2,1306.44,64.0,0.30008


In [21]:
all(frontier_pcd_df['Training time (chip hours)'].notna())

True

# Price selection

1. Use a fixed mapping from Organization to cloud provider. If no mapping found, default to "Amazon Web Services".
2. If there's a match for the hardware model, use that. Else, discard the ML system from the dataset.
3. Use the price that is nearest to, but prior to, training time + 2 months before the publication date
4. If there are no prices prior to that time, use the nearest price after that time
5. If there are no prices for that hardware model and cloud provider at all, repeat steps 3 and 4 for "Microsoft Azure", then "Google Cloud" as the cloud provider.
6. If there are no prices found from step 5, discard the ML system from the dataset.

In [22]:
# Example usage
example_vendor = "Google Cloud"
example_hardware_model = "Google TPU v3"
example_date = "2019-07-15" # Example date, format should be YYYY-MM-DD

# Find the row
closest_row_df = find_closest_price_dates(example_vendor, example_hardware_model, example_date, price_df)

for i, row in closest_row_df.iterrows():
    if row['Price date'] <= pd.to_datetime(example_date):
        print(f"Price date: {row['Price date']}")
        print(f"Price: {row['Price per chip-hour (1-year CUD)']}")
        break    

Price date: 2019-07-01 00:00:00
Price: $1.26


In [23]:
# TODO: use the vendor mapping from imputation to reduce repetition
org_to_cloud_vendor = {
    'google': 'Google Cloud',
    'deepmind': 'Google Cloud',
    'microsoft': 'Microsoft Azure',
    'openai': 'Microsoft Azure',
}

In [24]:
frontier_pcd_df['System'] = frontier_pcd_df.index

In [25]:
# Function to convert float year to datetime
def float_year_to_datetime(float_year):
    year = int(float_year)
    remainder = float_year - year
    days_in_year = 365 + int(pd.Timestamp(year=year, month=12, day=31).is_leap_year)
    day_of_year = int(remainder * days_in_year)
    return pd.Timestamp(year=year, month=1, day=1) + pd.to_timedelta(day_of_year, unit='D')

frontier_pcd_df['Publication date'] = frontier_pcd_df['Publication date'].apply(float_year_to_datetime)

In [26]:
pcd_hardware_model_colname = 'Training hardware'

In [27]:
price_colname = 'Price per chip-hour (1-year CUD)'
system_to_price = {}

for i, row in frontier_pcd_df.iterrows():
    price = find_price(row, price_df, hardware_df, pcd_hardware_model_colname, price_colname, org_to_cloud_vendor)
    if price is None:
        continue
    else:
        system_to_price[row['System']] = price

system_to_price

==== System: AlphaGo Fan ====
Trying Google TPU v3 at 2015-07-17 09:00:00
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 2.0

==== System: AlphaGo Lee ====
Trying Google TPU v3 at 2015-11-12 09:00:00
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 2.0

==== System: GNMT ====
Trying NVIDIA Tesla K80 at 2016-01-30 00:00:00
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 1.4

==== System: NASv3 (CIFAR-10) ====
Trying Google TPU v3 at 2016-08-21 09:00:00
Trying Google Cloud, Price per chip-hour

{'AlphaGo Fan': 2.0,
 'AlphaGo Lee': 2.0,
 'GNMT': 1.4,
 'NASv3 (CIFAR-10)': 2.0,
 'JFT': 1.4,
 'AlphaZero': 1.13,
 'AmoebaNet-A (F=448)': 0.052760065856088875,
 'IMPALA': 1.46,
 'ResNeXt-101 32x48d': 2.29,
 'FTW': 2.0,
 'BigGAN-deep 512x512': 2.0,
 'RoBERTa Large': 2.29,
 'Megatron-BERT': 0.1051277082983274,
 'Megatron-LM (8.3B)': 2.29,
 'T5-3B': 1.26,
 'T5-11B': 1.26,
 'AlphaStar': 1.26,
 'OpenAI Five Rerun': 2.29,
 'Meena': 1.26,
 'Turing-NLG': 2.29,
 'GPT-3 175B (davinci)': 2.29,
 'GShard (dense)': 1.26,
 'ALIGN': 1.26,
 'Megatron-Turing NLG 530B': 2.62,
 'Gopher (280B)': 1.26,
 'GLaM': 2.03,
 'LaMDA': 1.26,
 'Chinchilla': 2.03,
 'PaLM (540B)': 2.03,
 'OPT-175B': 3.0,
 'Minerva (540B)': 2.03,
 'GPT-3.5 (text-davinci-003)': 2.4,
 'LLaMA-65B': 2.35,
 'GPT-4': 2.4,
 'PaLM 2': 2.03,
 'Llama 2-70B': 3.0,
 'Falcon 180B': 2.4}

# Cost estimation

TODO: inflation adjustment

cost = price_per_chip_hour * chip_hours

In [28]:
def estimate_cost(row, system_to_price):
    system = row['System']
    price = system_to_price.get(system)
    if price is None:
        return None

    chip_hours = row['Training time (chip hours)']
    if np.isnan(chip_hours):
        return None

    cost = price * chip_hours

    # Check for base model
    if not pd.isna(row['Base model']):
        base_model_name = row['Base model']
        base_model = frontier_pcd_df[frontier_pcd_df['System'] == base_model_name].squeeze()
        base_cost = estimate_cost(base_model, system_to_price)
        if base_cost is None:
            return None
        else:
            cost += base_cost

    return cost
    

In [29]:
system_to_cost = {}
for i, row in frontier_pcd_df.iterrows():
    cost = estimate_cost(row, system_to_price)
    if cost is None:
        continue
    system_to_cost[row['System']] = cost

system_to_cost

{'AlphaGo Fan': 233674.55999999997,
 'AlphaGo Lee': 233674.55999999997,
 'GNMT': 580608.0,
 'NASv3 (CIFAR-10)': 600320.0,
 'JFT': 100800.0,
 'AlphaZero': 94481.7408,
 'AmoebaNet-A (F=448)': 3988.660978720319,
 'IMPALA': 146.0,
 'ResNeXt-101 32x48d': 962569.4400000001,
 'FTW': 1112889.6,
 'BigGAN-deep 512x512': 24576.0,
 'RoBERTa Large': 281395.2,
 'Megatron-BERT': 74924.93821505114,
 'Megatron-LM (8.3B)': 383400.96,
 'T5-3B': 147214.9728,
 'T5-11B': 310883.328,
 'AlphaStar': 510935.04,
 'OpenAI Five Rerun': 1466772.48,
 'Meena': 928972.8,
 'Turing-NLG': 605269.3504,
 'GPT-3 175B (davinci)': 8134080.0,
 'GShard (dense)': 1300561.92,
 'ALIGN': 224050.176,
 'Megatron-Turing NLG 530B': 9037952.0,
 'Gopher (280B)': 4748083.2,
 'GLaM': 2839531.5199999996,
 'LaMDA': 1786982.4,
 'Chinchilla': 3513681.2031999994,
 'PaLM (540B)': 17062133.759999998,
 'OPT-175B': 2437632.0,
 'Minerva (540B)': 18508922.88,
 'GPT-3.5 (text-davinci-003)': 21853839.36,
 'LLaMA-65B': 2406400.0,
 'GPT-4': 136800000.0,


In [30]:
for system, cost in system_to_cost.items():
    frontier_pcd_df.loc[system, 'Cost'] = cost

# Apply inflation adjustment

In [32]:
from_year_month = frontier_pcd_df['Publication date'].apply(str)
frontier_pcd_df['Publication date'] = from_year_month

In [33]:
frontier_pcd_df['Publication date']

System
AlphaGo Fan                   2015-10-01 00:00:00
AlphaGo Lee                   2016-01-27 00:00:00
GNMT                          2016-09-26 00:00:00
NASv3 (CIFAR-10)              2016-11-05 00:00:00
AlphaGo Master                2017-01-01 00:00:00
JFT                           2017-07-11 00:00:00
OpenAI TI7 DOTA 1v1           2017-08-11 00:00:00
AlphaGo Zero                  2017-10-18 00:00:00
AlphaZero                     2017-12-05 00:00:00
AmoebaNet-A (F=448)           2018-02-04 00:00:00
IMPALA                        2018-02-04 00:00:00
ResNeXt-101 32x48d            2018-05-03 00:00:00
FTW                           2018-07-04 00:00:00
BigGAN-deep 512x512           2018-09-28 00:00:00
RoBERTa Large                 2019-07-02 00:00:00
Megatron-BERT                 2019-09-17 00:00:00
Megatron-LM (8.3B)            2019-09-17 00:00:00
T5-3B                         2019-10-23 00:00:00
T5-11B                        2019-10-23 00:00:00
AlphaStar                     2019-10-30 00

In [34]:
frontier_pcd_df = adjust_column_for_inflation(frontier_pcd_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

# Regression

# Plots

In [35]:
import plotly.graph_objects as go

fig = go.Figure()
for i, row in frontier_pcd_df.iterrows():
    system = row['System']
    cost = system_to_cost.get(system)
    if cost is None:
        continue
    publication_date = row['Publication date']

    fig.add_trace(go.Scatter(
        x=[publication_date],
        y=[cost],
        name=system,
        text=system,
        textposition='top center',
        line=dict(color='#034752'),
        mode='markers+text',
    ))

# log y axis
fig.update_yaxes(type="log")
# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (USD, nominal)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
# fig.update_xaxes(range=['2017-01-01', '2025-01-01'])
# fig.update_yaxes(range=[5, 8])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [36]:
import plotly.graph_objects as go

fig = go.Figure()
for i, row in frontier_pcd_df.iterrows():
    system = row['System']
    cost = row['Cost (inflation-adjusted)']
    if cost is None:
        continue
    publication_date = row['Publication date']

    fig.add_trace(go.Scatter(
        x=[publication_date],
        y=[cost],
        name=system,
        text=system,
        textposition='top center',
        line=dict(color='#034752'),
        mode='markers+text',
    ))

# log y axis
fig.update_yaxes(type="log")
# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
# fig.update_xaxes(range=['2017-01-01', '2025-01-01'])
# fig.update_yaxes(range=[5, 8])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter-inflation_adjusted')

fig.show()

In [37]:
import plotly.graph_objects as go

fig = go.Figure()
for i, row in frontier_pcd_df.iterrows():
    system = row['System']
    cost = system_to_cost.get(system)
    if cost is None:
        continue
    publication_date = row['Publication date']

    fig.add_trace(go.Bar(
        x=[system],
        y=[cost],
        name=system,
        # nice blue color
        marker_color='#034752',
        # text=system,
        # textposition='auto',
    ))

# log y axis
fig.update_yaxes(type="log")
# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Cost (USD, nominal)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_yaxes(range=[0, 8])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

fig.show()


In [38]:
import plotly.graph_objects as go

fig = go.Figure()
for i, row in frontier_pcd_df.iterrows():
    system = row['System']
    cost = row['Cost (inflation-adjusted)']
    if cost is None:
        continue
    publication_date = row['Publication date']

    fig.add_trace(go.Bar(
        x=[system],
        y=[cost],
        name=system,
        # nice blue color
        marker_color='#034752',
        # text=system,
        # textposition='auto',
    ))

# log y axis
fig.update_yaxes(type="log")
# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_yaxes(range=[0, 8])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'costs-inflation_adjusted')

fig.show()


# Export data

In [39]:
frontier_pcd_df.drop('Megatron-BERT', inplace=True)
frontier_pcd_df.drop('IMPALA', inplace=True)

In [40]:
frontier_pcd_df.to_csv(results_dir + 'price dataset.csv')